#Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools, which pymc3 provides a wrapper `model.profile` which returns a `ProfileStats` object. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [1]:
import numpy as np
from pymc3 import *
from pymc3.math import exp
from pymc3.distributions.timeseries import *

n = 400
returns = np.genfromtxt(get_data('SP500.csv'))[-n:]
with Model() as model:
    sigma = Exponential('sigma', 1. / .02, testval=.1)
    nu = Exponential('nu', 1. / 10)
    s = GaussianRandomWalk('s', sigma ** -2, shape=n)
    r = StudentT('r', nu, lam=exp(-2 * s), observed=returns)
    


Then call profile and summarize it.

In [2]:
model.profile(model.logpt).summary()


Function profiling
  Message: /home/jovyan/pymc3/pymc3/model.py:605
  Time in 1000 calls to Function.__call__: 1.775136e-01s
  Time in Function.fn.__call__: 1.416550e-01s (79.800%)
  Time in thunks: 8.041668e-02s (45.302%)
  Total compile time: 1.353232e+00s
    Number of Apply nodes: 20
    Theano Optimizer time: 6.614311e-01s
       Theano validate time: 4.212379e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 6.327283e-01s
       Import time 3.420997e-02s
       Node make_thunk time 6.312668e-01s
           Node Elemwise{Composite{(Switch(Identity(GT(i0, i1)), (i2 - (i3 * i0)), i4) + i5 + Switch(Identity(GT(i6, i1)), (i7 - (i8 * i6)), i4) + i9 + i10 + i11)}}[(0, 0)](Elemwise{exp,no_inplace}.0, TensorConstant{0}, TensorConstant{3.9120230674743652}, TensorConstant{50.0}, TensorConstant{-inf}, sigma_log_, Elemwise{exp,no_inplace}.0, TensorConstant{-2.3025850929940455}, TensorConstant{0.1}, nu_log_, Sum{acc_dtype=float64}.0, Sum{acc_dtype=float64}.0) time 5.813

In [3]:
model.profile(gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /home/jovyan/pymc3/pymc3/model.py:605
  Time in 1000 calls to Function.__call__: 3.743136e-01s
  Time in Function.fn.__call__: 3.272467e-01s (87.426%)
  Time in thunks: 1.778915e-01s (47.525%)
  Total compile time: 1.396206e+00s
    Number of Apply nodes: 47
    Theano Optimizer time: 6.084559e-01s
       Theano validate time: 1.443505e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 7.295318e-01s
       Import time 8.256626e-02s
       Node make_thunk time 7.264183e-01s
           Node Elemwise{Composite{Switch(i0, (i1 * (i2 + ((i3 * i4 * i5 * i6) / i7))), i8)}}[(0, 6)](Elemwise{Composite{Identity((GT(i0, i1) * i2 * GT(inv(sqrt(i0)), i1)))}}.0, TensorConstant{(1,) of -2.0}, TensorConstant{(1,) of 0.5}, TensorConstant{(1,) of -0.5}, InplaceDimShuffle{x}.0, TensorConstant{[  4.05769..48400e-06]}, Elemwise{Composite{exp((i0 * i1))}}.0, Elemwise{Add}[(0, 1)].0, TensorConstant{(1,) of 0}) time 5.903370e-01s
           Node Join(TensorC